In [1]:
class Layer:
    def __init__(self) -> None:
        self.input=None
        self.outpt=None

    def forward_propagation(self,input):
        raise NotImplementedError
    def backPropagation(self,output_error,learning_rate):
        raise NotImplementedError

In [2]:
import numpy as np
class FCLayer(Layer):
    def __init__(self,input_size,output_size) -> None:
        self.weights=np.random.rand(input_size,output_size)-.5
        self.bias=np.random.rand(1,output_size)-.5
    def forward_propagation(self, input):
        self.input=input
        self.outpt=np.dot(self.input,self.weights)+self.bias
        return self.outpt
    def backPropagation(self, output_error, learning_rate):
        dEdX=np.dot(output_error,self.weights.T)
        dEdW=np.dot(self.input.T,output_error)
        self.weights-=learning_rate*dEdW
        self.bias-=learning_rate*output_error
        return dEdX
        
    

In [3]:
class ActivationLayer(Layer):
    def __init__(self,activation,activation_prime) -> None:
        self.activation=activation
        self.activation_prime=activation_prime
    def forward_propagation(self, input):
        self.input=input
        self.output=self.activation(self.input)
        return self.output
    def backPropagation(self, output_error, learning_rate):
        return self.activation_prime(self.input)*output_error

In [4]:
def tanh(x):
    return np.tanh(x)

def tanh_prime(x):
    return 1-np.tanh(x)**2

def sigmoid(x):
    return np.exp(x)

def sigmoid_prime(x):
    return sigmoid(x)*(1-sigmoid(x))

def atan(x):
    return np.arctan(x)
def atan_prime(x):
    return 1/(x**2+1)

In [5]:
def mse(y_true, y_pred):
    return np.mean(np.power(y_true-y_pred, 2))

def mse_prime(y_true, y_pred):
    return 2*(y_pred-y_true)/y_true.size

In [6]:
class NeuralNetwork:
    def __init__(self,loss,lossPrime):
        self.layers:list[Layer]=[]
        self.loss =loss
        self.lossPrime=lossPrime
    def addLayer(self,layer:Layer):
        self.layers.append(layer)
        

    def train(self,x_train,y_train,epochs=40,learning_rate=0.1):
        samples=len(x_train)
        for n in range(epochs):
            currentError=0
            for j in range(samples):
                output=x_train[j]
                for layer in self.layers:
                    output=layer.forward_propagation(output)
                currentError+=self.loss(y_train[j],output)

                outputError=self.lossPrime(y_train[j],output)
                for layer in reversed(self.layers):
                    outputError=layer.backPropagation(outputError,learning_rate)
            currentError/=samples
            print(f'epoch= {n} error={currentError}')

    def predict(self,x_test):
        samples=len(x_test)
        res=[]
        for n in range(samples):
            output=x_test[n]
            for layer in self.layers:
                output=layer.forward_propagation(output)
            res.append(output)
        return res
    


In [7]:
network=NeuralNetwork(mse,mse_prime)
network.addLayer(FCLayer(3,5))
network.addLayer(ActivationLayer(tanh,tanh_prime))
network.addLayer(FCLayer(5,7))
network.addLayer(ActivationLayer(tanh,tanh_prime))
network.addLayer(FCLayer(7,9))
network.addLayer(ActivationLayer(tanh,tanh_prime))
network.addLayer(FCLayer(9,14))
network.addLayer(ActivationLayer(atan,atan_prime))
network.addLayer(FCLayer(14,20))
network.addLayer(ActivationLayer(atan,atan_prime))

network.addLayer(FCLayer(20,15))
network.addLayer(ActivationLayer(atan,atan_prime))
network.addLayer(FCLayer(15,10))
network.addLayer(ActivationLayer(atan,atan_prime))
network.addLayer(FCLayer(10,8))
network.addLayer(ActivationLayer(atan,atan_prime))
network.addLayer(FCLayer(8,6))
network.addLayer(ActivationLayer(tanh,tanh_prime))
network.addLayer(FCLayer(6,3))
network.addLayer(ActivationLayer(tanh,tanh_prime))

In [8]:
x_train = np.array([[[0,0,0]], [[0,1,0]], [[1,0,1]], [[1,1,0]],[[0,1,1]],[[0,0,1]]])
y_train = np.array([[[0,0,1]], [[0,1,1]], [[1,1,0]], [[1,1,1]],[[1,0,0]],[[0,1,0]]])

In [9]:
network.train(x_train,y_train,epochs=6000,learning_rate=0.2)

epoch= 0 error=0.3893904991484951
epoch= 1 error=0.28974605611985693
epoch= 2 error=0.287206754165304
epoch= 3 error=0.28610274597239477
epoch= 4 error=0.2851199154057909
epoch= 5 error=0.2842157373468846
epoch= 6 error=0.28339719152524995
epoch= 7 error=0.2826570920398131
epoch= 8 error=0.2819822787654233
epoch= 9 error=0.2813594734221892
epoch= 10 error=0.28077723219674217
epoch= 11 error=0.28022619419035644
epoch= 12 error=0.2796987482812578
epoch= 13 error=0.27918856397688346
epoch= 14 error=0.27869014103087525
epoch= 15 error=0.2781984121041851
epoch= 16 error=0.2777083823313224
epoch= 17 error=0.2772147671742154
epoch= 18 error=0.27671157489773085
epoch= 19 error=0.2761915594415293
epoch= 20 error=0.2756454309658247
epoch= 21 error=0.2750606360748681
epoch= 22 error=0.2744193709883744
epoch= 23 error=0.27369519029004824
epoch= 24 error=0.27284694699915396
epoch= 25 error=0.271807457680077
epoch= 26 error=0.27046143441154485
epoch= 27 error=0.26860199763574566
epoch= 28 error=0.26

In [10]:
network.predict([[1,0,0]])

[array([[0.89792997, 0.99980814, 0.99448279]])]